In [2]:
import numpy as np
import pandas as pd
import re
from ethnicolr2 import pred_fl_last_name, pred_fl_full_name
import os
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [11]:
os.cpu_count()

384

Reference: 

https://github.com/appeler/ethnicolr2?tab=readme-ov-file

https://github.com/appeler/ethnicolr_v2?tab=readme-ov-file


In [6]:
df_single = pd.read_csv("./race_gender_data/df_single.csv").iloc[:, 1:]
df_others = pd.read_csv("./race_gender_data/df_others.csv").iloc[:, 1:]

In [2]:
df = pd.read_csv("./race_gender_data/df.csv").iloc[:, 1:]

In [5]:
df

,firstname,lastname,country,fullname
0,sabouni,manal,FR,sabouni manal
1,claudia,perretta,UK,claudia perretta
2,yass,yass,CA,yass
3,tina,mccoy,US,tina mccoy
4,whitney,reed,US,whitney reed
...,...,...,...,...
1907993,tanya,ashworth,AU,tanya ashworth
1907994,chris,kowalenko,UK,chris kowalenko
1907995,faith,cummings,US,faith cummings
1907996,karen,van,US,karen van


In [7]:
print(df.shape, df_single.shape, df_others.shape)

NameError: name 'df' is not defined

In [8]:
df_single

,firstname,lastname,country,fullname,pred_gender
0,yass,yass,CA,yass,Female
1,zak,zak,UK,zak,Female
2,alyssa,alyssa,US,alyssa,Male
3,kyra,kyra,UK,kyra,Female
4,aftondivver,aftondivver,US,aftondivver,Male
...,...,...,...,...,...
65984,iturospee,iturospee,US,iturospee,Male
65985,hernatkova,hernatkova,US,hernatkova,Female
65986,dyce,dyce,US,dyce,Female
65987,jamal,jamal,DE,jamal,Male


In [9]:
df_others

,firstname,lastname,country,fullname,pred_gender
0,sabouni,manal,FR,sabouni manal,Male
1,claudia,perretta,UK,claudia perretta,Female
2,tina,mccoy,US,tina mccoy,Female
3,whitney,reed,US,whitney reed,Female
4,leonora,camovic,US,leonora camovic,Female
...,...,...,...,...,...
1646836,tanya,ashworth,AU,tanya ashworth,Female
1646837,chris,kowalenko,UK,chris kowalenko,Male
1646838,faith,cummings,US,faith cummings,Female
1646839,karen,van,US,karen van,Female


In [11]:
df

,firstname,lastname,country,fullname
0,sabouni,manal,FR,sabouni manal
1,claudia,perretta,UK,claudia perretta
2,yass,yass,CA,yass yass
3,tina,mccoy,US,tina mccoy
4,whitney,reed,US,whitney reed
...,...,...,...,...
1907993,tanya,ashworth,AU,tanya ashworth
1907994,chris,kowalenko,UK,chris kowalenko
1907995,faith,cummings,US,faith cummings
1907996,karen,van,US,karen van


In [5]:
df_gender = pd.read_csv("./race_gender_data/df_gender.csv")
unknown_entries = df_gender[df_gender["pred_gender"] == "Unknown"]
print("Entries with Unknown predicted gender:")
print(unknown_entries)
df_gender

Entries with Unknown predicted gender:
Empty DataFrame
Columns: [firstname, lastname, country, fullname, pred_gender]
Index: []


,firstname,lastname,country,fullname,pred_gender
0,sabouni,manal,FR,sabouni manal,Male
1,claudia,perretta,UK,claudia perretta,Female
2,yass,yass,CA,yass,Female
3,tina,mccoy,US,tina mccoy,Female
4,whitney,reed,US,whitney reed,Female
...,...,...,...,...,...
1712825,tanya,ashworth,AU,tanya ashworth,Female
1712826,chris,kowalenko,UK,chris kowalenko,Male
1712827,faith,cummings,US,faith cummings,Female
1712828,karen,van,US,karen van,Female


# Race

In [10]:
df_others

,firstname,lastname,country,fullname,pred_gender
0,sabouni,manal,FR,sabouni manal,Male
1,claudia,perretta,UK,claudia perretta,Female
2,tina,mccoy,US,tina mccoy,Female
3,whitney,reed,US,whitney reed,Female
4,leonora,camovic,US,leonora camovic,Female
...,...,...,...,...,...
1646836,tanya,ashworth,AU,tanya ashworth,Female
1646837,chris,kowalenko,UK,chris kowalenko,Male
1646838,faith,cummings,US,faith cummings,Female
1646839,karen,van,US,karen van,Female


# fullname+df_others

In [3]:
df_others = pd.read_csv("./race_gender_data/df_others.csv")
df_others

,Unnamed: 0,firstname,lastname,country,fullname,pred_gender
0,0,sabouni,manal,FR,sabouni manal,Male
1,1,claudia,perretta,UK,claudia perretta,Female
2,2,tina,mccoy,US,tina mccoy,Female
3,3,whitney,reed,US,whitney reed,Female
4,4,leonora,camovic,US,leonora camovic,Female
...,...,...,...,...,...,...
1646836,1646836,tanya,ashworth,AU,tanya ashworth,Female
1646837,1646837,chris,kowalenko,UK,chris kowalenko,Male
1646838,1646838,faith,cummings,US,faith cummings,Female
1646839,1646839,karen,van,US,karen van,Female


## test with chunk

### save chunk csv files - works

In [ ]:
def process_in_chunks(df, chunk_size):
    results = []
    for start in range(0, len(df), chunk_size):
        end = min(start + chunk_size, len(df))
        chunk = df.iloc[start:end]

        # Run prediction on the chunk
        pred_chunk = pred_fl_full_name(
            chunk, lname_col="lastname", fname_col="firstname"
        )

        # Save the current chunk to a CSV file
        chunk_number = start // chunk_size + 1
        pred_chunk.to_csv(
            f"./race_gender_data/df_others_chunk_{chunk_number}.csv", index=False
        )
    print(f"Processed chunk from {start} to {end}")


pred_with_fullname = process_in_chunks(df_others, chunk_size=500000)
pred_with_fullname.to_csv("./race_gender_data/race_gender_full_others.csv", index=False)

In [ ]:
# Define a function to process in chunks
def process_in_chunks(df, chunk_size):
    results = []
    for start in range(0, len(df), chunk_size):
        end = min(start + chunk_size, len(df))
        chunk = df.iloc[start:end]

        # Run prediction on the chunk
        pred_chunk = pred_fl_full_name(
            chunk, lname_col="lastname", fname_col="firstname"
        )
        results.append(pred_chunk)

    # Concatenate all results
    return pd.concat(results, ignore_index=True)
    print(f"Processed chunk from {start} to {end}")


# Process df_others in chunks of 1000 rows
pred_with_fullname = process_in_chunks(df_others, chunk_size=500000)

# Save the concatenated results to CSV
pred_with_fullname.to_csv("./race_gender_data/race_gender_full_others.csv", index=False)

## test

In [4]:
pred_with_fullname = pred_fl_full_name(
    df_others[:500000], lname_col="lastname", fname_col="firstname"
)
pred_with_fullname.to_csv("./race_gender_data/race_gender_full_others.csv")

100%|██████████| 7813/7813 [03:49<00:00, 34.08it/s]


In [6]:
pred_with_fullname = pred_fl_full_name(
    df_others[1000000:], lname_col="lastname", fname_col="firstname"
)
pred_with_fullname.to_csv("./race_gender_data/df_other_chunk_3.csv")

  0%|          | 28/10107 [00:00<03:31, 47.74it/s]


IndexError: index out of range in self

# lastname +df_single


## Read df_single

In [22]:
df_single = pd.read_csv("./race_gender_data/df_single.csv")
df_single

,Unnamed: 0,firstname,lastname,country,fullname,pred_gender
0,0,yass,yass,CA,yass,Female
1,1,zak,zak,UK,zak,Female
2,2,alyssa,alyssa,US,alyssa,Male
3,3,kyra,kyra,UK,kyra,Female
4,4,aftondivver,aftondivver,US,aftondivver,Male
...,...,...,...,...,...,...
65984,65984,iturospee,iturospee,US,iturospee,Male
65985,65985,hernatkova,hernatkova,US,hernatkova,Female
65986,65986,dyce,dyce,US,dyce,Female
65987,65987,jamal,jamal,DE,jamal,Male


## test with chunk, df_others+df_single

In [ ]:
# Define a function to process in chunks
def process_in_chunks(
    df, chunk_size, prediction_function, lname_col=None, fname_col=None
):
    results = []
    for start in range(0, len(df), chunk_size):
        end = min(start + chunk_size, len(df))
        chunk = df.iloc[start:end]

        # Run prediction on the chunk
        if prediction_function == pred_fl_full_name:
            pred_chunk = prediction_function(
                chunk, lname_col=lname_col, fname_col=fname_col
            )
        elif prediction_function == pred_fl_last_name:
            pred_chunk = prediction_function(chunk, lname_col=lname_col)

        results.append(pred_chunk)

    # Concatenate all results
    return pd.concat(results, ignore_index=True)
    print(f"Processed chunk from {start} to {end}")


# Process df_others in chunks of 1000 rows
pred_with_fullname = process_in_chunks(
    df_others,
    chunk_size=100000,
    prediction_function=pred_fl_full_name,
    lname_col="lastname",
    fname_col="firstname",
)

# Save the concatenated results to CSV
pred_with_fullname.to_csv("./race_gender_data/race_gender_full_others.csv", index=False)

# Process df_single in chunks for last name prediction
pred_with_lastname = process_in_chunks(
    df_single,
    chunk_size=5000,
    prediction_function=pred_fl_last_name,
    lname_col="lastname",
)
pred_with_lastname.to_csv("./race_gender_data/race_gender_last_single.csv", index=False)

### save chunk csv files

In [23]:
# Define a function to process the dataframe in chunks and save each chunk
def process_in_chunks_lastname(df, chunk_size):
    results = []
    for start in range(0, len(df), chunk_size):
        end = min(start + chunk_size, len(df))
        chunk = df.iloc[start:end]

        # Run prediction on the chunk
        pred_chunk = pred_fl_last_name(chunk, lname_col="lastname")
        results.append(pred_chunk)

        # Save the current chunk to a CSV file
        chunk_number = start // chunk_size + 1
        pred_chunk.to_csv(
            f"./race_gender_data/df_single_chunk_{chunk_number}.csv", index=False
        )
        print(f"Processed chunk from {start} to {end}")

    # Concatenate all results
    return pd.concat(results, ignore_index=True)


pred_fl_last_name = process_in_chunks(df_single, chunk_size=10000)
pred_fl_last_name.to_csv("./race_gender_data/race_gender_last_.csv", index=False)

 67%|██████▋   | 105/157 [00:02<00:01, 35.88it/s]


IndexError: index out of range in self

## test

In [10]:
pred_fl_last_name(df_single, lname_col="lastname")

100%|██████████| 1/1 [00:00<00:00, 10.28it/s]


,last,first,true_race,preds,probs
0,yass,yass yass,nh_white,nh_white,"{'asian': 0.039757933, 'hispanic': 0.39860517,..."
1,raul,raul,hispanic,nh_white,"{'asian': 0.017201504, 'hispanic': 0.023858989..."
2,xx,xx,hispanic,nh_white,"{'asian': 0.012569758, 'hispanic': 0.026675707..."
3,X,RoneDanielFastFloBgi,hispanic,asian,"{'asian': 0.70105106, 'hispanic': 0.02619504, ..."


# Result

Concat chunk files

In [1]:
import pandas as pd
import glob


# Concatenate all df_others chunks
def concat_chunks(file_pattern, output_file):
    # Use glob to match the file pattern and load all matching files
    chunk_files = glob.glob(file_pattern)

    # Concatenate all chunks
    df_combined = pd.concat(
        [pd.read_csv(file) for file in chunk_files], ignore_index=True
    )

    # Save the combined dataframe to a new CSV file
    df_combined.to_csv(output_file, index=False)
    print(f"Concatenated {len(chunk_files)} files into {output_file}")


# Concatenate df_others chunks
concat_chunks(
    "./race_gender_data/df_others_chunk_*.csv",
    "./race_gender_data/df_others_race_gender.csv",
)

# Concatenate df_single chunks
concat_chunks(
    "./race_gender_data/df_single_chunk_*.csv",
    "./race_gender_data/df_single_race_gender.csv",
)

Concatenated 4 files into ./race_gender_data/df_others_race_gender.csv
Concatenated 7 files into ./race_gender_data/df_single_race_gender.csv


## Clean race_gender_last_single

In [12]:
race_gender_last_single = pd.read_csv("./race_gender_data/df_single_race_gender.csv")
# # remove the last column
# race_gender_last_single = race_gender_last_single.iloc[:, :-1]
race_gender_last_single

,firstname,lastname,country,fullname,pred_gender,preds,probs
0,yass,yass,CA,yass,Female,nh_white,"{'asian': 0.039757933, 'hispanic': 0.39860517,..."
1,zak,zak,UK,zak,Female,nh_white,"{'asian': 0.022933554, 'hispanic': 0.020127507..."
2,zak,zak,UK,zak,Female,nh_white,"{'asian': 0.022933554, 'hispanic': 0.020127507..."
3,zak,zak,UK,zak,Female,nh_white,"{'asian': 0.022933554, 'hispanic': 0.020127507..."
4,alyssa,alyssa,US,alyssa,Male,nh_white,"{'asian': 0.029911637, 'hispanic': 0.1969176, ..."
...,...,...,...,...,...,...,...
78764,fregouville,fregouville,FR,fregouville,Male,nh_white,"{'asian': 0.004903849, 'hispanic': 0.067926176..."
78765,msahli,msahli,FR,msahli,Male,nh_white,"{'asian': 0.07436386, 'hispanic': 0.049667317,..."
78766,boccheciampe,boccheciampe,FR,boccheciampe,Female,hispanic,"{'asian': 0.023130896, 'hispanic': 0.77701527,..."
78767,jordpulkownik,jordpulkownik,US,jordpulkownik,Male,nh_white,"{'asian': 0.0054982407, 'hispanic': 0.03553804..."


In [13]:
race_gender_last_single = race_gender_last_single.dropna()
race_gender_last_single = race_gender_last_single.drop_duplicates()
race_gender_last_single

,firstname,lastname,country,fullname,pred_gender,preds,probs
0,yass,yass,CA,yass,Female,nh_white,"{'asian': 0.039757933, 'hispanic': 0.39860517,..."
1,zak,zak,UK,zak,Female,nh_white,"{'asian': 0.022933554, 'hispanic': 0.020127507..."
4,alyssa,alyssa,US,alyssa,Male,nh_white,"{'asian': 0.029911637, 'hispanic': 0.1969176, ..."
6,kyra,kyra,UK,kyra,Female,nh_white,"{'asian': 0.029823758, 'hispanic': 0.016653847..."
10,aftondivver,aftondivver,US,aftondivver,Male,nh_white,"{'asian': 0.015394655, 'hispanic': 0.040740304..."
...,...,...,...,...,...,...,...
78764,fregouville,fregouville,FR,fregouville,Male,nh_white,"{'asian': 0.004903849, 'hispanic': 0.067926176..."
78765,msahli,msahli,FR,msahli,Male,nh_white,"{'asian': 0.07436386, 'hispanic': 0.049667317,..."
78766,boccheciampe,boccheciampe,FR,boccheciampe,Female,hispanic,"{'asian': 0.023130896, 'hispanic': 0.77701527,..."
78767,jordpulkownik,jordpulkownik,US,jordpulkownik,Male,nh_white,"{'asian': 0.0054982407, 'hispanic': 0.03553804..."


## Clean race_gender_full_others

In [14]:
race_gender_full_others = pd.read_csv("./race_gender_data/df_others_race_gender.csv")

In [15]:
race_gender_full_others

,firstname,lastname,country,fullname,pred_gender,preds,probs
0,sabouni,manal,FR,sabouni manal,Male,nh_black,"{'asian': 0.090085596, 'hispanic': 0.011093363..."
1,claudia,perretta,UK,claudia perretta,Female,nh_white,"{'asian': 0.0024087005, 'hispanic': 0.23162283..."
2,tina,mccoy,US,tina mccoy,Female,nh_white,"{'asian': 0.0040824614, 'hispanic': 0.00605579..."
3,whitney,reed,US,whitney reed,Female,nh_white,"{'asian': 0.0016876273, 'hispanic': 0.00515399..."
4,leonora,camovic,US,leonora camovic,Female,nh_white,"{'asian': 0.0365716, 'hispanic': 0.016523963, ..."
...,...,...,...,...,...,...,...
1690178,lucy,youd,UK,lucy youd,Female,nh_white,"{'asian': 0.033374153, 'hispanic': 0.07930497,..."
1690179,nadine,jessup,US,nadine jessup,Male,nh_black,"{'asian': 0.0071938434, 'hispanic': 0.01306549..."
1690180,leyton,attwood,UK,leyton attwood,Male,nh_white,"{'asian': 0.004400566, 'hispanic': 0.009662661..."
1690181,marcus,henthorn,US,marcus henthorn,Male,nh_white,"{'asian': 0.0031183357, 'hispanic': 0.01353228..."


In [16]:
race_gender_full_others = race_gender_full_others.dropna()
race_gender_full_others = race_gender_full_others.drop_duplicates()
race_gender_full_others

,firstname,lastname,country,fullname,pred_gender,preds,probs
0,sabouni,manal,FR,sabouni manal,Male,nh_black,"{'asian': 0.090085596, 'hispanic': 0.011093363..."
1,claudia,perretta,UK,claudia perretta,Female,nh_white,"{'asian': 0.0024087005, 'hispanic': 0.23162283..."
2,tina,mccoy,US,tina mccoy,Female,nh_white,"{'asian': 0.0040824614, 'hispanic': 0.00605579..."
3,whitney,reed,US,whitney reed,Female,nh_white,"{'asian': 0.0016876273, 'hispanic': 0.00515399..."
4,leonora,camovic,US,leonora camovic,Female,nh_white,"{'asian': 0.0365716, 'hispanic': 0.016523963, ..."
...,...,...,...,...,...,...,...
1690178,lucy,youd,UK,lucy youd,Female,nh_white,"{'asian': 0.033374153, 'hispanic': 0.07930497,..."
1690179,nadine,jessup,US,nadine jessup,Male,nh_black,"{'asian': 0.0071938434, 'hispanic': 0.01306549..."
1690180,leyton,attwood,UK,leyton attwood,Male,nh_white,"{'asian': 0.004400566, 'hispanic': 0.009662661..."
1690181,marcus,henthorn,US,marcus henthorn,Male,nh_white,"{'asian': 0.0031183357, 'hispanic': 0.01353228..."


In [20]:
result_df = pd.concat([race_gender_last_single, race_gender_full_others], axis=0)
result_df

,firstname,lastname,country,fullname,pred_gender,preds,probs
0,yass,yass,CA,yass,Female,nh_white,"{'asian': 0.039757933, 'hispanic': 0.39860517,..."
1,zak,zak,UK,zak,Female,nh_white,"{'asian': 0.022933554, 'hispanic': 0.020127507..."
4,alyssa,alyssa,US,alyssa,Male,nh_white,"{'asian': 0.029911637, 'hispanic': 0.1969176, ..."
6,kyra,kyra,UK,kyra,Female,nh_white,"{'asian': 0.029823758, 'hispanic': 0.016653847..."
10,aftondivver,aftondivver,US,aftondivver,Male,nh_white,"{'asian': 0.015394655, 'hispanic': 0.040740304..."
...,...,...,...,...,...,...,...
1690178,lucy,youd,UK,lucy youd,Female,nh_white,"{'asian': 0.033374153, 'hispanic': 0.07930497,..."
1690179,nadine,jessup,US,nadine jessup,Male,nh_black,"{'asian': 0.0071938434, 'hispanic': 0.01306549..."
1690180,leyton,attwood,UK,leyton attwood,Male,nh_white,"{'asian': 0.004400566, 'hispanic': 0.009662661..."
1690181,marcus,henthorn,US,marcus henthorn,Male,nh_white,"{'asian': 0.0031183357, 'hispanic': 0.01353228..."


In [21]:
result_df.to_csv("./username_gender_race.csv")

In [23]:
pd.read_csv("username_gender_race.csv")

,Unnamed: 0,firstname,lastname,country,fullname,pred_gender,preds,probs
0,0,yass,yass,CA,yass,Female,nh_white,"{'asian': 0.039757933, 'hispanic': 0.39860517,..."
1,1,zak,zak,UK,zak,Female,nh_white,"{'asian': 0.022933554, 'hispanic': 0.020127507..."
2,4,alyssa,alyssa,US,alyssa,Male,nh_white,"{'asian': 0.029911637, 'hispanic': 0.1969176, ..."
3,6,kyra,kyra,UK,kyra,Female,nh_white,"{'asian': 0.029823758, 'hispanic': 0.016653847..."
4,10,aftondivver,aftondivver,US,aftondivver,Male,nh_white,"{'asian': 0.015394655, 'hispanic': 0.040740304..."
...,...,...,...,...,...,...,...,...
1712797,1690178,lucy,youd,UK,lucy youd,Female,nh_white,"{'asian': 0.033374153, 'hispanic': 0.07930497,..."
1712798,1690179,nadine,jessup,US,nadine jessup,Male,nh_black,"{'asian': 0.0071938434, 'hispanic': 0.01306549..."
1712799,1690180,leyton,attwood,UK,leyton attwood,Male,nh_white,"{'asian': 0.004400566, 'hispanic': 0.009662661..."
1712800,1690181,marcus,henthorn,US,marcus henthorn,Male,nh_white,"{'asian': 0.0031183357, 'hispanic': 0.01353228..."
